# What
分类任务，支持两种模式
1. Folder模式，需要输入`train`, `valid`两个测试集对应的目录。`labels.txt`，需要训练的label，里面每个类别一行。
2. List模式，需要输入`train`, `valid`两个测试集对应的训练文件，每行一个样本。`labels.txt`是可选参数，里面每个类别一行。`data_pattern`一个通用的目录，与train、val中的第一列进行拼接。

In [ ]:
import os
from onekey_algo.classification.run_classification import main as clf_main
from collections import namedtuple
from onekey_algo import get_param_in_cwd

# 设置参数
root = get_param_in_cwd('radio_dir')
for model_name in ['resnet18', 'resnet50', 'resnet101', 'densenet121', 'vgg19', 'densenet201', 'inception_v3'][:1]:
    for cv in range(0, 1):
        train_f = os.path.join('split_info', f'train-RND-{cv}.txt')
        val_f = os.path.join('split_info', f'val-RND-{cv}.txt')
        labels_f = os.path.join('split_info', 'labels.txt')
        data_pattern = os.path.join(root, f'crop')
        params = dict(train=train_f,
                      valid=val_f,
                      labels_file=labels_f,
                      in_channels=3,
                      data_pattern=data_pattern,
                      j=6,
                      max2use=None,
                      val_max2use=None,
                      batch_balance=False,
                      normalize_method='imagenet',
                      model_name=model_name,
                      gpus=[0],
                      batch_size=32,
                      epochs=64 if model_name not in ['TwinsSVT', 'CrossFormer', 'SimpleViT'] else 96,
                      init_lr=0.01 if model_name not in ['TwinsSVT', 'CrossFormer', 'SimpleViT'] else 0.001,
                      optimizer='sgd',
                      retrain=r'',
                      model_root= os.path.join(root, 'models', f"CV-{cv}"),
                      iters_start=0,
                      iters_verbose=16,
                      save_per_epoch=True,
                      add_date=False,
                      pretrained=False)
        # 训练模型
        Args = namedtuple("Args", params)
        clf_main(Args(**params))